# Email Agent with Tools

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from crewai import LLM
import os

llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.1, api_key=os.getenv("GOOGLE_API_KEY"))


# Building Tools using CrewAI BaseTools

In [16]:
from crewai.tools import BaseTool

class ReplaceAbbreviationsTool(BaseTool):
    name: str = "Jargon replacement tool"
    description : str = "Replaces jargon with more specific terms. "

    def _run(self, email: str) -> str:
        replacements = {
            "PRX": "Project Phoenix (internal AI revamp project)",
            "TAS": "technical architecture stack",
            "DBX": "client database cluster",
            "SDS": "Smart Data Syncer",
            "SYNCBOT": "internal standup assistant bot",
            "WIP": "in progress",
            "POC": "proof of concept",
            "ping": "reach out"
        }
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")

        return "\n".join(suggestions) if suggestions else "No jargon or internal abbreviations detected."

jt = ReplaceAbbreviationsTool()

original_email = """
looping in Praveen. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.
"""

jt.run(original_email)

Using Tool: Jargon replacement tool


"Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'\nConsider replacing 'TAS' with 'technical architecture stack'\nConsider replacing 'SDS' with 'Smart Data Syncer'\nConsider replacing 'SYNCBOT' with 'internal standup assistant bot'\nConsider replacing 'ping' with 'reach out'"

# Add tools to the agent

In [18]:
from crewai import Agent, Task, Crew

email_assistant = Agent(
    role="Email Assistant Agent",
    goal="Improve emails and make them sound professional and clear",
    backstory="A highly experienced communication expert skilled in professional email writing",
    verbose=True,
    tools=[jt], #jt is the abbreviation replacement tool
    llm=llm
)

email_task = Task(
    description=f"""Take the following rough email and rewrite it into a professional and polished version.
    Expand abbreviations:
    '''{original_email}'''""", # original_email is the rough email
    agent=email_assistant,
    expected_output="A professional written email with proper formatting and content and should be in email format.",
)

crew = Crew(
    agents=[email_assistant],
    tasks=[email_task],
    verbose=True
)

result = crew.kickoff()
print(result)

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 2fa98789-ff7f-4c74-b440-67b52f177029                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 28cb00c3-1c9a-46c3-94ef-ab8d35f991a1
    Status: Executing Task...
╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Email A

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x146a714d0>, 'Connection to telemetry.crewai.com timed out. (connect timeout=29.999999046325684)'))
